# Chapter 2: Working with text data

## 2.2 Tokenizing text

In [ ]:
with open("the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()
print(f'total number of characters: {len(raw_text)}')
print(raw_text[:99])

In [ ]:
import re # regular expressions

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
result

In [ ]:
result = re.split(r'([,.]|\s)', text)
result

In [ ]:
result = [item for item in result if item.strip()]
result

In [ ]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(len(preprocessed))

In [ ]:
preprocessed[:30]

## 2.3 Converting tokens into token IDs

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

In [ ]:
vocab = {token: integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text) # removes spaces before the punctuation characters
        return text

In [ ]:
tokenizer = SimpleTokenizerV1(vocab=vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

In [ ]:
ids

In [ ]:
tokenizer.decode(ids)

## 2.4 Adding special context tokens

In [ ]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

In [ ]:
vocab = {token: integer for integer,token in enumerate(all_tokens)}

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" 
                        for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text) # removes spaces before the punctuation characters
        return text

In [ ]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab=vocab)
print(tokenizer.encode(text))

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.5 Byte pair encoding

In [ ]:
from importlib.metadata import version
import tiktoken
version("tiktoken")

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)
integers = tokenizer.encode(text=text, allowed_special={"<|endoftext|>"})
integers

In [ ]:
strings = tokenizer.decode(integers)
strings

In [ ]:
text = ("xu")
integers = tokenizer.encode(text=text, allowed_special={"<|endoftext|>"})
strings = tokenizer.decode(integers)
strings

In [ ]:
integers

## 2.6 Data sampling

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
len(enc_text)

In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
x,y

In [ ]:
for i in range(1,context_size+1):
    input_seq = enc_sample[:i]
    target = enc_sample[i]
    print(input_seq,"->",target)

In [ ]:
for i in range(1,context_size+1):
    input_seq = enc_sample[:i]
    target = enc_sample[i]
    print(tokenizer.decode(input_seq),"->",tokenizer.decode([target]))

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # super().__init__()
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length,stride)
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers) # number of CPU processes
    return dataloader

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(
    txt=raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)
data_iter = iter(dataloader) # so we can use the Python built-in function next()
first_batch = next(data_iter)
first_batch

In [ ]:
dataloader = create_dataloader_v1(
    txt=raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
inputs, targets

## 2.7 Create token embeddings

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

In [ ]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
embedding_layer.weight

In [ ]:
embedding_layer.weight.shape

In [ ]:
embedding_layer(torch.tensor([3]))

In [ ]:
embedding_layer(input_ids)

## 2.8 Encoding word positions

In [ ]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, 
    max_length=max_length, stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

In [ ]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings

In [ ]:
pos_embeddings.shape # 4 vectors, one for each position in the context window

In [ ]:
input_embeddings = token_embeddings + pos_embeddings # works for the entire batch, cuz python broadcasting
input_embeddings.shape